# সেমান্টিক কোর 

এই কোড নমুনায়, আপনি [সেমান্টিক কোর](https://aka.ms/ai-agents-beginners/semantic-kernel) এআই ফ্রেমওয়ার্ক ব্যবহার করে একটি সাধারণ এজেন্ট তৈরি করবেন। 

এই নমুনার লক্ষ্য হলো আপনাকে সেই ধাপগুলো দেখানো, যা আমরা পরে বিভিন্ন এজেন্টিক প্যাটার্ন বাস্তবায়নের সময় অতিরিক্ত কোড নমুনাগুলিতে ব্যবহার করব।


## প্রয়োজনীয় পাইথন প্যাকেজগুলি ইমপোর্ট করুন


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## ক্লায়েন্ট তৈরি করা

এই উদাহরণে, আমরা [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ব্যবহার করব LLM-এ অ্যাক্সেস পাওয়ার জন্য।

`ai_model_id` হিসেবে `gpt-4o-mini` নির্ধারণ করা হয়েছে। GitHub Models মার্কেটপ্লেসে উপলব্ধ অন্য কোনো মডেলে পরিবর্তন করে দেখুন বিভিন্ন ফলাফল কেমন হয়।

`Azure Inference SDK` ব্যবহার করার জন্য, যা GitHub Models-এর `base_url` এর জন্য ব্যবহৃত হয়, আমরা Semantic Kernel-এর মধ্যে `OpenAIChatCompletion` সংযোগকারী ব্যবহার করব। এছাড়াও অন্যান্য [উপলব্ধ সংযোগকারী](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) রয়েছে যা অন্যান্য মডেল প্রদানকারীদের জন্য Semantic Kernel ব্যবহার করতে সাহায্য করে।


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## এজেন্ট তৈরি করা

নিচে আমরা `TravelAgent` নামে একটি এজেন্ট তৈরি করছি।

এই উদাহরণের জন্য, আমরা খুব সাধারণ নির্দেশাবলী ব্যবহার করছি। আপনি এই নির্দেশাবলী পরিবর্তন করতে পারেন এবং দেখতে পারেন এজেন্ট কীভাবে ভিন্নভাবে প্রতিক্রিয়া জানায়।


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## এজেন্ট চালানো

এখন আমরা `ChatHistoryAgentThread` টাইপের একটি থ্রেড সংজ্ঞায়িত করে এজেন্ট চালাতে পারি। এজেন্টের invoke_stream এর `messages` কীওয়ার্ড আর্গুমেন্টে প্রয়োজনীয় সিস্টেম বার্তাগুলি প্রদান করা হয়।

এগুলো সংজ্ঞায়িত করার পর, আমরা একটি `user_inputs` তৈরি করি যা ব্যবহারকারী এজেন্টকে পাঠাবে। এই ক্ষেত্রে, আমরা এই বার্তাটি `Plan me a sunny vacation` সেট করেছি।

আপনি চাইলে এই বার্তাটি পরিবর্তন করতে পারেন এবং দেখতে পারেন এজেন্ট কীভাবে ভিন্নভাবে প্রতিক্রিয়া জানায়।


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিক অনুবাদ প্রদানের চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
